In [1]:
#importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [2]:
#loading in the main dataset
df = pd.read_csv("cleaned_dataset.csv")

In [3]:
#setting pandas to show all columns of the dataframe without truncation
pd.set_option('display.max_columns', None)

In [4]:
#checking the main dataframe
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Time_bin,Manufactured_year,Age,Year_month,Date_time,Delay_status
0,2007,9,27,4,1104,1105,1243,1255,WN,110,N741SA,99.0,110.0,82.0,-12.0,-1.0,SEA,SMF,605,5,12,0,0,0,0,0,10am - 12pm,1998,9,2007-09,2007-09-27 11:05:00,0
1,2007,11,20,2,2120,2120,2255,2308,EV,4304,N355CA,95.0,108.0,67.0,-13.0,0.0,ATL,CLE,554,10,18,0,0,0,0,0,8pm - 10pm,2002,5,2007-11,2007-11-20 21:20:00,0
2,2007,7,16,1,1025,1030,1127,1133,OO,5799,N284YV,62.0,63.0,54.0,-6.0,-5.0,PDX,MFR,222,2,6,0,0,0,0,0,10am - 12pm,1994,13,2007-07,2007-07-16 10:30:00,0
3,2007,8,5,7,1448,1453,1529,1538,AQ,94,N837AL,41.0,45.0,32.0,-9.0,-5.0,HNL,KOA,163,2,7,0,0,0,0,0,2pm - 4pm,1985,22,2007-08,2007-08-05 14:53:00,0
4,2006,12,10,7,2027,1921,2235,2138,OO,6704,N943SW,68.0,77.0,55.0,57.0,66.0,DEN,LNK,423,3,10,0,0,0,0,57,6pm - 8pm,2003,3,2006-12,2006-12-10 19:21:00,1


### Q4) Can you detect cascading failures as delays in one airport create delays in others?

In [5]:
#keeping only the necessary columns
q4 = df.loc[:, ["Year", "Month", "DayofMonth", "DayOfWeek","CRSDepTime", "CRSArrTime", "FlightNum", "TailNum",
                "ArrDelay", "Origin", "Dest", "Date_time"]]

In [6]:
#sorting by tail number and then Date time, and setting Date_time as the index
q4 = q4.sort_values(["TailNum", "Date_time"]).set_index("Date_time")

In [7]:
#having a look at the new Date_time Column
q4.head()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,FlightNum,TailNum,ArrDelay,Origin,Dest
Date_time,,,,,,,,,,,
2006-01-01 16:20:00,2006,1,1,7,1620,1820,2352,N10156,-9.0,RDU,IAH
2006-01-02 14:35:00,2006,1,2,1,1435,1653,2172,N10156,-3.0,RIC,IAH
2006-01-03 10:25:00,2006,1,3,2,1025,1345,3023,N10156,60.0,IAH,CLT
2006-01-03 14:15:00,2006,1,3,2,1415,1605,3023,N10156,363.0,CLT,EWR
2006-01-04 06:50:00,2006,1,4,3,650,810,2399,N10156,67.0,ORF,EWR


In [8]:
#creating the Next_delay column by shifting the current ArrDelay 1 row up
q4["Next_delay"] = q4["ArrDelay"].shift(-1)

In [9]:
#checking the new Next_delay column
q4.head(10)

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,FlightNum,TailNum,ArrDelay,Origin,Dest,Next_delay
Date_time,,,,,,,,,,,,
2006-01-01 16:20:00,2006,1,1,7,1620,1820,2352,N10156,-9.0,RDU,IAH,-3.0
2006-01-02 14:35:00,2006,1,2,1,1435,1653,2172,N10156,-3.0,RIC,IAH,60.0
2006-01-03 10:25:00,2006,1,3,2,1025,1345,3023,N10156,60.0,IAH,CLT,363.0
2006-01-03 14:15:00,2006,1,3,2,1415,1605,3023,N10156,363.0,CLT,EWR,67.0
2006-01-04 06:50:00,2006,1,4,3,650,810,2399,N10156,67.0,ORF,EWR,16.0
2006-01-04 09:35:00,2006,1,4,3,935,1144,2932,N10156,16.0,EWR,STL,-11.0
2006-01-06 08:20:00,2006,1,6,5,820,1050,2184,N10156,-11.0,PIT,IAH,4.0
2006-01-06 19:20:00,2006,1,6,5,1920,2250,2185,N10156,4.0,IAH,DAY,3.0
2006-01-07 16:15:00,2006,1,7,6,1615,2008,2771,N10156,3.0,IAH,ORF,-26.0


In [10]:
#checking for null values
q4.isnull().sum()

Year          0
Month         0
DayofMonth    0
DayOfWeek     0
CRSDepTime    0
CRSArrTime    0
FlightNum     0
TailNum       0
ArrDelay      0
Origin        0
Dest          0
Next_delay    1
dtype: int64

In [11]:
#dropping the remaining null values
q4.dropna(inplace=True)

In [12]:
#checking the number of rows of the dataframe
q4.shape

(5750845, 12)

In [13]:
#encoding the arrival delay and next delay columns, to HasCurrentDelay and HasNextDelay respectively
q4["HadCurrentDelay"] = q4["ArrDelay"].apply(lambda x: 1 if x>0 else 0)
q4["HasNextDelay"] = q4["Next_delay"].apply(lambda x: 1 if x>0 else 0)

In [14]:
#having a look at the dataframe
q4.head()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,FlightNum,TailNum,ArrDelay,Origin,Dest,Next_delay,HasCurrentDelay,HasNextDelay
Date_time,,,,,,,,,,,,,,
2006-01-01 16:20:00,2006,1,1,7,1620,1820,2352,N10156,-9.0,RDU,IAH,-3.0,0,0
2006-01-02 14:35:00,2006,1,2,1,1435,1653,2172,N10156,-3.0,RIC,IAH,60.0,0,1
2006-01-03 10:25:00,2006,1,3,2,1025,1345,3023,N10156,60.0,IAH,CLT,363.0,1,1
2006-01-03 14:15:00,2006,1,3,2,1415,1605,3023,N10156,363.0,CLT,EWR,67.0,1,1
2006-01-04 06:50:00,2006,1,4,3,650,810,2399,N10156,67.0,ORF,EWR,16.0,1,1


In [15]:
#creating the contingency table 
crosstab = pd.crosstab(q4["HadNextDelay"], q4["HasCurrentDelay"], rownames=["Had current delay"], 
                       colnames=["Has next delay"]).apply(lambda x:x/x.sum(),axis=0)
crosstab.head() 

Has next delay,0,1
Had current delay,,
0,0.625768,0.422979
1,0.374232,0.577021


If a plane had an arrival delay at the current airport it is likely that plane will experience an arrival delay at the next airport as well.